LendingClubCaseStudy

In [45]:
import pandas as pd

Reading the Dataset

In [46]:
df=pd.read_csv('loan.csv',low_memory=False)
df.shape

(39717, 111)

Cleaning the dataset

In [47]:
#Checking the percentage of Nan values in dataset
percentage_null=round(df.isnull().sum() * 100 / len(df),2)
percentage_null

id                              0.0
member_id                       0.0
loan_amnt                       0.0
funded_amnt                     0.0
funded_amnt_inv                 0.0
                              ...  
tax_liens                       0.1
tot_hi_cred_lim               100.0
total_bal_ex_mort             100.0
total_bc_limit                100.0
total_il_high_credit_limit    100.0
Length: 111, dtype: float64

In [48]:
#Droping the Columns having greater or equal to 75% Nan values
df=df.drop(percentage_null[percentage_null>=75.0].index,axis=1)
df.shape

(39717, 55)

In [42]:
# Finding the columns having unique data
# Sorted unique values based on the columns
# Dropping the unnecessary columns as per the column's data uniqueness
# dropped columns [tax_liens, delinq_amnt, chargeoff_within_12_mths, acc_now_delinq, application_type, policy_code, collections_12_mths_ex_med, initial_list_status pymnt_plan]
df.drop(df.nunique().sort_values().head(9).index, axis=1, inplace=True)

In [43]:
df.shape

(39717, 46)

In [71]:
s=df.nunique().sort_values()
df1 = s.to_frame().reset_index()
df1.rename(columns = {'index':'names',0:'uniquecount'},inplace = True)

In [81]:
df1=df1[df1.uniquecount==1]
df1
type(df1)

pandas.core.frame.DataFrame

In [75]:
df.drop(columns=df1[names],axis=1)

NameError: name 'names' is not defined

In [ ]:
df.shape